# This is a student solution in TDT05 2020
## The notebook is read-only because the data files are not available

In [2]:
# Import and init h2o lib. Running on java
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment AdoptOpenJDK (build 11.0.10+9); OpenJDK 64-Bit Server VM AdoptOpenJDK (build 11.0.10+9, mixed mode)
  Starting server from /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/y1/6h3k81fn12g2nt2tpwdrrkn40000gn/T/tmpfmbc15e5
  JVM stdout: /var/folders/y1/6h3k81fn12g2nt2tpwdrrkn40000gn/T/tmpfmbc15e5/h2o_adrianlangseth_started_from_python.out
  JVM stderr: /var/folders/y1/6h3k81fn12g2nt2tpwdrrkn40000gn/T/tmpfmbc15e5/h2o_adrianlangseth_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_adrianlangseth_dxmhym
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [12]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [3]:
# Reading csv to h2o dataframe file
df = h2o.import_file('../tdt05-2021-challenge-2/challenge2_train.csv')
df_test = h2o.import_file('../tdt05-2021-challenge-2/challenge2_test.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [42]:
# Reading csv to h2o dataframe file
train = pd.read_csv('../tdt05-2021-challenge-2/challenge2_train.csv')
test = pd.read_csv('../tdt05-2021-challenge-2/challenge2_test.csv')

ord_features = ['f1', 'f2', 'f3', 'f5', 'f7', 'f10', 'f13', 'f18', 'f19', 'f27']
num_ords = ['f3', 'f5', 'f7', 'f19', 'f27']
alpha_ords = ['f1', 'f2', 'f10', 'f18']

numeric = ['f11', 'f17', 'f24', 'f28']
bell_curve = ['f11', 'f28']
long_tail = ['f17', 'f24']  # F24 has -1 as null values. Remove before scaling

bin_features = ['f0', 'f4', 'f6', 'f25', 'f26']
cyc_features = ['f16', 'f21']
nom_features = ['f8', 'f9', 'f12', 'f14', 'f15', 'f22', 'f23']  # hexes
duplicate = ['f20']

all_feat=bin_features+nom_features+ord_features+['f16_sin', 'f16_cos', 'f21_sin', 'f21_cos']+numeric

categorical = bin_features+nom_features+ord_features+ ['f1_0', 'f1_1']+['f16_sin', 'f16_cos', 'f21_sin', 'f21_cos']

#Split up the dub
train['f1_0'] = train['f1'].apply(lambda x: x[0] if type(x) is str else x)
test['f1_0'] = test['f1'].apply(lambda x: x[0] if type(x) is str else x)
train['f1_1'] = train['f1'].apply(lambda x: x[1] if type(x) is str else x)
test['f1_1'] = test['f1'].apply(lambda x: x[1] if type(x) is str else x)
train.drop(['f1'], axis=1, inplace=True)
test.drop(['f1'], axis=1, inplace=True)
ord_features.remove('f1')
ord_features.extend(['f1_0', 'f1_1'])

# EHM something

train[['f5']].replace(-1.0, np.nan, inplace=True)
test[['f5']].replace(-1.0, np.nan, inplace=True)

# Convert those alphas which are very indicative

train['f13'] = train['f13'].apply(lambda x: (ord(x) - ord('a'))/14 if pd.notnull(x) else x)
test['f13'] = test['f13'].apply(lambda x: (ord(x) - ord('a'))/14 if pd.notnull(x) else x)
ord_features.remove('f13')

train['f10'] = train['f10'].apply(lambda x: (ord(x) - ord('A'))/25 if pd.notnull(x) else x)
test['f10'] = test['f10'].apply(lambda x: (ord(x) - ord('A'))/25 if pd.notnull(x) else x)
ord_features.remove('f10')


## ChooChoo here comes the number soup
scl = preprocessing.StandardScaler()
train[['f11', 'f28']] = scl.fit_transform(train[['f11', 'f28']])
test[['f11', 'f28']] = scl.transform(test[['f11', 'f28']])
    
train[['f24']].replace(-1.0, np.nan, inplace=True)
test[['f24']].replace(-1.0, np.nan, inplace=True)
train[['f17', 'f24']] = np.log(train[['f17', 'f24']])
test[['f17', 'f24']] = np.log(test[['f17', 'f24']])

scl = preprocessing.StandardScaler()
train[['f17', 'f24']] = scl.fit_transform(train[['f17', 'f24']])
test[['f17', 'f24']] = scl.transform(test[['f17', 'f24']])

# 9 million bicycles in Beijing

train['f16_sin'] = np.sin((train['f16'] - 1) * (2. * np.pi / 12))
train['f16_cos'] = np.cos((train['f16'] - 1) * (2. * np.pi / 12))
test['f16_sin'] = np.sin((test['f16'] - 1) * (2. * np.pi / 12))
test['f16_cos'] = np.cos((test['f16'] - 1) * (2. * np.pi / 12))
train.drop(['f16'], axis=1, inplace=True)
test.drop(['f16'], axis=1, inplace=True)

train['f21_sin'] = np.sin((train['f21'] - 1) * (2. * np.pi / 7))
train['f21_cos'] = np.cos((train['f21'] - 1) * (2. * np.pi / 7))
test['f21_sin'] = np.sin((test['f21'] - 1) * (2. * np.pi / 7))
test['f21_cos'] = np.cos((test['f21'] - 1) * (2. * np.pi / 7))
train.drop(['f21'], axis=1, inplace=True)
test.drop(['f21'], axis=1, inplace=True)
    
train['f16_sin'].fillna(0, inplace=True)
train['f16_cos'].fillna(0, inplace=True)
test['f16_sin'].fillna(0, inplace=True)
test['f16_cos'].fillna(0, inplace=True)

train['f21_sin'].fillna(0, inplace=True)
train['f21_cos'].fillna(0, inplace=True)
test['f21_sin'].fillna(0, inplace=True)
test['f21_cos'].fillna(0, inplace=True)

train['f19'] = train['f19'].apply(lambda x: int(x*10) if pd.notnull(x) else x)
test['f19'] = test['f19'].apply(lambda x: int(x*10) if pd.notnull(x) else x)

target = train['target']
test_id = test['id']
#train.drop(['target', 'id', 'f20'], axis=1, inplace=True)
#test.drop(['id', 'f20'], axis=1, inplace=True)
train.drop(['id', 'f20'], axis=1, inplace=True)
test.drop(['id', 'f20'], axis=1, inplace=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in log
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:56: RuntimeWarning: invalid value encount

In [43]:
categorical = bin_features+nom_features+ord_features # + ['f1_0', 'f1_1']# +['f16_sin', 'f16_cos', 'f21_sin', 'f21_cos']
categorical.sort()
print(categorical)

['f0', 'f12', 'f14', 'f15', 'f18', 'f19', 'f1_0', 'f1_1', 'f2', 'f22', 'f23', 'f25', 'f26', 'f27', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9']


In [44]:
train.head().T

,0,1,2,3,4
target,0,0,0,1,0
f0,1,0,NaN,1,1
f2,e,c,a,c,c
f3,3,1,1,3,3
f4,A,A,A,A,A
f5,NaN,7,10,7,7
f6,0,1,1,1,0
f7,6,4,6,1,6
f8,96ae67d3e,9fcf422f2,a5adff44e,15c90ab2e,b36490559
f9,yellow,green,white,yellow,yellow


In [45]:
train.f13.unique()
cats = ['target', 'f0', 'f2', 'f3', 'f4', 'f4', 'f4' ]

In [91]:
h_train = h2o.H2OFrame(train)
h_test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [92]:
for col in categorical:
    h_train[col] = h_train[col].asfactor()
    h_test[col] = h_test[col].asfactor()


h_train['target'] = h_train['target'].asfactor()

In [93]:
# Removing labels from train set
y = "target"
x = h_train.columns
x.remove(y)

In [94]:
# Running fit/training on train set with cross validation n=5
aml = H2OAutoML(max_models = 20, seed = 42)
aml.train(x = x, y = y, training_frame = h_train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_7_AutoML_7_20211021_121433

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.09379269011090771
RMSE: 0.30625592257278506
LogLoss: 0.30721997468114776
Null degrees of freedom: 9963
Residual degrees of freedom: 9947
Null deviance: 9660.898713412476
Residual deviance: 6122.279655445911
AIC: 6156.279655445911
AUC: 0.9111452070883735
AUCPR: 0.7397162780620231
Gini: 0.8222904141767471

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.33680456902342076: 


,,0,1,Error,Rate
0,0,7389.0,692.0,0.0856,(692.0/8081.0)
1,1,579.0,1304.0,0.3075,(579.0/1883.0)
2,Total,7968.0,1996.0,0.1276,(1271.0/9964.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.336805,0.672338,169.0
1,max f2,0.205900,0.747714,239.0
2,max f0point5,0.466948,0.701153,108.0
3,max accuracy,0.411546,0.879968,134.0
4,max precision,0.856591,1.000000,0.0
5,max recall,0.026764,1.000000,375.0
6,max specificity,0.856591,1.000000,0.0
7,max absolute_mcc,0.336805,0.593615,169.0
8,max min_per_class_accuracy,0.242119,0.825393,219.0
9,max mean_per_class_accuracy,0.236782,0.829004,222.0



Gains/Lift Table: Avg response rate: 18,90 %, avg score: 19,27 %


...

,,0,1,Error,Rate
0,0,31907.0,8837.0,0.2169,(8837.0/40744.0)
1,1,3706.0,5550.0,0.4004,(3706.0/9256.0)
2,Total,35613.0,14387.0,0.2509,(12543.0/50000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.240609,0.469484,208.0
1,max f2,0.111725,0.608057,306.0
2,max f0point5,0.352437,0.444398,141.0
3,max accuracy,0.511290,0.823140,69.0
4,max precision,0.817980,1.000000,0.0
5,max recall,0.007494,1.000000,397.0
6,max specificity,0.817980,1.000000,0.0
7,max absolute_mcc,0.240609,0.328349,208.0
8,max min_per_class_accuracy,0.193807,0.697403,240.0
9,max mean_per_class_accuracy,0.186203,0.698920,246.0



Gains/Lift Table: Avg response rate: 18,51 %, avg score: 18,51 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.01,0.622201,3.586863,3.586863,0.6640,0.677025,0.664000,0.677025,0.035869,0.035869,258.686258,258.686258,0.031745
1,2,0.02,0.573344,3.305964,3.446413,0.6120,0.596413,0.638000,0.636719,0.033060,0.068928,230.596370,244.641314,0.060044
2,3,0.03,0.539960,3.143907,3.345578,0.5820,0.556630,0.619333,0.610023,0.031439,0.100367,214.390666,234.557764,0.086353
3,4,0.04,0.511368,2.981850,3.254646,0.5520,0.525237,0.602500,0.588826,0.029818,0.130186,198.184961,225.464564,0.110674
4,5,0.05,0.487556,2.517286,3.107174,0.4660,0.498887,0.575200,0.570838,0.025173,0.155359,151.728608,210.717373,0.129293
5,6,0.10,0.402991,2.350908,2.729041,0.4352,0.441301,0.505200,0.506070,0.117545,0.272904,135.090752,172.904062,0.212183
6,7,0.15,0.346069,2.059205,2.505762,0.3812,0.373404,0.463867,0.461848,0.102960,0.375864,105.920484,150.576203,0.277175
7,8,0.20,0.300958,1.754538,2.317956,0.3248,0.322401,0.429100,0.426986,0.087727,0.463591,75.453760,131.795592,0.323472
8,9,0.30,0.232699,1.503889,2.046600,0.2784,0.265092,0.378867,0.373021,0.150389,0.613980,50.388937,104.660040,0.385308
9,10,0.40,0.182562,1.098747,1.809637,0.2034,0.206460,0.335000,0.331381,0.109875,0.723855,9.874676,80.963699,0.397426


In [95]:
lb = aml.leaderboard

In [96]:
# Showing best perforing models
lb.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_7_AutoML_7_20211021_121433,0.767583,0.40698,0.427979,0.30864,0.358633,0.128617
StackedEnsemble_AllModels_4_AutoML_7_20211021_121433,0.767455,0.407044,0.427717,0.307317,0.358682,0.128653
StackedEnsemble_AllModels_3_AutoML_7_20211021_121433,0.767395,0.407128,0.427694,0.307849,0.358727,0.128685
StackedEnsemble_BestOfFamily_4_AutoML_7_20211021_121433,0.766446,0.407581,0.427615,0.310161,0.358857,0.128778
StackedEnsemble_BestOfFamily_7_AutoML_7_20211021_121433,0.766293,0.407758,0.426999,0.304314,0.358931,0.128831
StackedEnsemble_AllModels_2_AutoML_7_20211021_121433,0.765079,0.40847,0.42518,0.308378,0.359303,0.129099
StackedEnsemble_BestOfFamily_3_AutoML_7_20211021_121433,0.764727,0.408589,0.425156,0.310448,0.359347,0.12913
StackedEnsemble_AllModels_6_AutoML_7_20211021_121433,0.764089,0.409463,0.418786,0.310178,0.359874,0.129509
StackedEnsemble_BestOfFamily_6_AutoML_7_20211021_121433,0.763252,0.409752,0.417696,0.310602,0.360002,0.129601
StackedEnsemble_AllModels_1_AutoML_7_20211021_121433,0.762602,0.409774,0.422933,0.312216,0.359774,0.129437


In [120]:
preds = aml.predict(h_test)

AttributeError: 'DataFrame' object has no attribute 'predict'

In [98]:
preds = h2o.as_list(preds)

In [99]:
preds = preds['p1']


In [100]:
path='../tdt05-2021-challenge-2/'
submission=pd.read_csv(path+'sample_submission.csv')

In [101]:
submission['target'] =preds #preds
submission.to_csv('results/automl.csv', index=None)
submission.head()

,target,id
0,0.349177,50000
1,0.467554,50001
2,0.177332,50002
3,0.357373,50003
4,0.140656,50004


In [106]:
submission=pd.read_csv(path+'sample_submission.csv')
own = pd.read_csv('results/all_feats_13_and_11_asnum_hyp_cat_cyclicals_nans_fixed_7_folds.csv')
aml = pd.read_csv('results/automl.csv')

In [118]:
submission['target'] = (aml.target*0.76772 + own.target*.76911)/(.76772+.76911)

In [119]:
submission.to_csv('results/aml_own_blend.csv', index=None)